# _P. californicus_ testes (pc85s5) analysis using Drop-seq core computational protocol

Monika Hrubá, Bulah Wu, Petr Nguyen

May 19, 2025

## Quality control for raw sequencing reads
FastQC is used for checking the quality of raw FASTQ files.  
Output can be found [here (read 1)](../shared/fastqc/pc85s5_r1_fastqc.html) and [here (read 2)](../shared/fastqc/pc85s5_r2_fastqc.html).

## Generating digital expression matrix
The raw reads are processed according to the [Drop-seq core computational protocol](https://github.com/broadinstitute/Drop-seq/blob/master/doc/Drop-seq_Alignment_Cookbook.pdf).  
The digital expression matrix is extracted by selecting cells with ≥ 20 UMI, as suggested by [James Nemesh](https://brain.broadinstitute.org/team/james_nemesh/)).

## Barcode rank plot
The barcode rank plot is used to assess the droplet data, with the knee and inflection points on the curve marking the transition between high RNA cells and empty droplets containing minimal RNA.  
These points are identified using the barcoderanks function from the DropletUtils R package.  
  
<img src="./files/pc85s5_barcoderanks.png" width="600">

## Empty droplet detection and filtering
The emptyDrops function from the DropletUtils R package is used to detect and remove empty droplets.  
The statistical test is executed with 10,000 iterations, and a False Discovery Rate (FDR) threshold of 0.001 is applied.

In [1]:
import pandas as pd
tbl_emptydrops = pd.read_csv("./files/pc85s5_tbl_emptydrops.tsv", sep="\t", header=0, index_col=0)
tbl_emptydrops

,FALSE,TRUE
False,4501,95
True,0,1612


In [2]:
true_cell = tbl_emptydrops.iloc[:, 1].sum()
print("Number of cells identified:", true_cell)

Number of cells identified: 1707


## Quality assessment of scRNA-seq cells
- nFeature_RNA: number of genes detected in each cell
    - Cells with very few genes detected might be empty droplets, dead cells, or cells with low RNA content.
    - Cells with an unexpectedly high number of detected genes might indicate multiplets or doublets.
- nCount_RNA: number of molecules (UMIs) captured per cell
    - Cells with very low nCount_RNA might be empty droplets or low-quality cells with minimal RNA capture.
    - Very high nCount_RNA might suggest multiplets or doublets.
- percent.mt: percentage of reads mapping to mitochondrial genes
    - A high percent.mt (often greater than 5-10%) could suggest low-quality cells that are dying or experiencing stress. This is because stressed or dying cells may express higher levels of mitochondrial genes due to cellular damage.
    - Cells with low percent.mt are generally of better quality.
  
We use violin plots with jittered data points to display nFeature_RNA, nCount_RNA, and percent.mt.

## Stepwise filtering of cells and genes
### Step 1: load the unfiltered digital expression matrix
<img src="./files/pc85s5_vln_s1.png" width="600">

### Step 2: apply emptyDrops to identify and select true cells
<img src="./files/pc85s5_vln_s2.png" width="600">

### Step 3: keep genes detected in a minimum of 3 cells
<img src="./files/pc85s5_vln_s3.png" width="600">

### Step 4: keep cells with more than 200 and less than 2500 detected genes
<img src="./files/pc85s5_vln_s4.png" width="600">

### Step 5: keep cells with mitochondrial percentages under 5%
<img src="./files/pc85s5_vln_s5.png" width="600">

### Summary
This table outlines, for each filtering step, the detected cell and gene counts, along with the mean and median UMI and gene numbers per cell.

In [3]:
summary_stat = pd.read_csv("./files/pc85s5_summary_stat.tsv", sep="\t", header=0)
summary_stat.index = ["Step 1", "Step 2", "Step 3", "Step 4", "Step 5"]
summary_stat

,NumGenes,NumCells,Mean_nCount_RNA,Median_nCount_RNA,Mean_nFeature_RNA,Median_nFeature_RNA
Step 1,9506,38306,161.639613,41.5,88.182086,38
Step 2,9506,1707,2359.422378,835.0,865.882250,534
Step 3,8385,1707,2358.737551,835.0,865.222613,534
Step 4,8385,1365,1659.922344,936.0,791.226374,589
Step 5,8385,1342,1676.837556,945.5,797.325633,592


## Performing data analysis with Seurat in R
The filtered matrix is loaded into Seurat, and the data is normalized, with mitochondrial percentage regressed out to avoid potential biases. Principal Component Analysis (PCA) is performed, followed by the identification of cell neighbors using the first 30 principal components. UMAP is applied to visualize the high-dimensional data in 2D, and cells are clustered using graph-based clustering.

### UMAP
<img src="./files/pc85s5_umap.png" width="600">